In [1]:
import ghmm
from collections import OrderedDict
import cPickle as pickle
import numpy as np
from itertools import product as iterproduct, chain
from pprint import pprint
import pysam
import os
import pandas
from copy import deepcopy
import re
import editdistance
import sys
import random
from nbwrapper import getargs
from multiprocessing import Pool
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/alignment_validation.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/alignment_lib.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/analysis_lib.ipynb"

In [2]:
args = getargs()

/home/ibis/gregor.sturm/bin/anaconda/lib/python2.7/site-packages/nbwrapper.py:31: RuntimeWarning: no arguments passed!
  warnings.warn("no arguments passed!", RuntimeWarning)


In [3]:
NMERS = 5

In [4]:
# args = {
#     "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_events.template.pickle",
#     "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_calling",
#     "ref": "/home/ibis/gregor.sturm/nanopore/david_eccles_bc_ideas/mouse_ref.fa",
#     "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_hmm_params_3mer.pickle",
#     "ncores": 4,
#     "nmers": 3,
#     "multivariate": True
# }

args = {
    "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_events.template.62pA.pickle",
    "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_calling",
    "ref": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_ref.fasta",
    "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_hmm_params.62pA.{0}mer.pickle".format(NMERS),
    "ncores": 62,
    "nmers": NMERS,
    "multivariate": False
}

In [5]:
NMERS = int(args["nmers"])
NSTATES = 4**NMERS
MULTIVARIATE = bool(int(args["multivariate"]))
args["ncores"] = int(args["ncores"])

In [6]:
HMM_PARAMS = pickle.load(open(args["hmm_params"], 'rb'))
HMM_PARAMS = OrderedDict(HMM_PARAMS)
ALL_KMERS = ["".join(x) for x in iterproduct("ACGT", repeat=NMERS)]
assert HMM_PARAMS.keys() == ALL_KMERS

# Train Model 

In [7]:
def mk_transmat1(nmers):
    """make a transition matrix assuming move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 1/4. if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)] else 0.
            transmat[j, i] = p          
            
    return transmat.tolist()

In [8]:
def mk_transmat0(nmers):
    """make a transition matrix assuming move=0 or move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (9/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [9]:
def mk_transmat2(nmers):
    """make a transition matrix assuming move=0 or move=1 or move=2"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-2):] == to_kmer[:(NMERS-2)]:
                """move=2"""
                p = (2/50.) * (1/16.)
            elif from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (47/50.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/50.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [10]:
mk_transmat = mk_transmat2

In [11]:
F = ghmm.Float()  # emission domain of this model

def mk_model_multivariate():
    # example code for a continuous HMM with gaussian emissions

    A = mk_transmat(NMERS)
    B = [ [ 
            [float(df[['mean']].mean()), float(df[['stdv']].mean())], #mu1, mu2
            [x for x in chain(*df[['mean', 'stdv']].cov().values.tolist())], #covariance matrix
        ] for df in HMM_PARAMS.values()]   # parameters of emission distributions in pairs of (mu, sigma)
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state

    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.MultivariateGaussianDistribution(F), A, B, pi)
    return model

In [12]:
def mk_model_simple(): 
    """ simple model, only taking the means into account. """
    A = mk_transmat(NMERS)
    B = [ [float(df[['mean']].mean()), float(df[['mean']].std())] #mu1, stdv
            for df in HMM_PARAMS.values()]   # parameters of emission distributions in pairs of (mu, sigma)
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state

    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.GaussianDistribution(F), A, B, pi)
    return model

In [13]:
def mk_model():
    if MULTIVARIATE: 
        return mk_model_multivariate()
    else: 
        return mk_model_simple()

In [14]:
model = mk_model()
s = str(model)
print(s)

GaussianEmissionHMM(N=1024)
  state 0 (initial=0.00, mu=65.44, sigma=1.18)
    Transitions: ->0 (0.00), ->1 (0.00), ->2 (0.00), ->3 (0.00), ->4 (0.00), ->5 (0.00), ->6 (0.00), ->7 (0.00), ->8 (0.00), ->9 (0.00), ->10 (0.00), ->11 (0.00), ->12 (0.00), ->13 (0.00), ->14 (0.00), ->15 (0.00)
  state 1 (initial=0.00, mu=61.47, sigma=1.08)
    Transitions: ->1 (0.02), ->4 (0.23), ->5 (0.23), ->6 (0.23), ->7 (0.23), ->16 (0.00), ->17 (0.00), ->18 (0.00), ->19 (0.00), ->20 (0.00), ->21 (0.00), ->22 (0.00), ->23 (0.00), ->24 (0.00), ->25 (0.00), ->26 (0.00), ->27 (0.00), ->28 (0.00), ->29 (0.00), ->30 (0.00), ->31 (0.00)

  ...

  state 1022 (initial=0.00, mu=47.86, sigma=0.81)
    Transitions: ->992 (0.00), ->993 (0.00), ->994 (0.00), ->995 (0.00), ->996 (0.00), ->997 (0.00), ->998 (0.00), ->999 (0.00), ->1000 (0.00), ->1001 (0.00), ->1002 (0.00), ->1003 (0.00), ->1004 (0.00), ->1005 (0.00), ->1006 (0.00), ->1007 (0.00), ->1016 (0.23), ->1017 (0.23), ->1018 (0.23), ->1019 (0.23), ->1022 (0.02)

In [15]:
def result_to_seq(result):
    states = result[0]
    kmers = [ALL_KMERS[x] for x in states]
    seq = [kmer[NMERS/2] for kmer in kmers]
    return "".join(seq)

In [16]:
def predict(mixed):
    """mixed is a set of tuples (event_mean, event_stdv)"""
    if MULTIVARIATE: 
        emissions = [x for x in chain(*mixed)]    
    else: 
        emissions = [x[0] for x in mixed]
    seq = ghmm.EmissionSequence(F, emissions)
    result = model.viterbi(seq)
    return result_to_seq(result)

In [17]:
s = model.sampleSingle(10)
s = [x for x in s]
seq = zip([s[i] for i in range(0, len(s), 2)], [s[i] for i in range(1, len(s), 2)])

In [18]:
predict(seq)

'ATCTG'

# Validate Model 

In [19]:
!pwd

/home/ibis/gregor.sturm/nanopore/own/notebooks


In [20]:
assert os.path.isfile(args["events"])

In [21]:
ref_file = args["ref"]
test = !cat {ref_file} | grep ">"
print(test)
ref = !cat {ref_file} | grep -v ">"
ref = ref[0]
print(ref[:100])

['>burn-in lambda_ref']
GGGCGGCGACCTCGCGGGTTTTCGCTATTTATGAAAATTTTCCGGTTTAAGGCGTTTCCGTTCTTCTTCGTCATAACTTAATGTTTTTATTTAAAATACC


In [22]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [23]:
prepare_filemap(file_data)

In [24]:
def train_read(file_obj):    
    events = [x["mean"] for x in file_obj["events"].to_dict("records")]
    seq = ghmm.EmissionSequence(F, events)
    model.baumWelch(seq)
    

In [25]:
def basecall_read(file_obj):
    events = [(x["mean"], x["stdv"]) for x in file_obj["events"].to_dict("records")]
    called_seq = predict(events)
    return (file_obj["channel"], file_obj["file_id"], called_seq)

In [26]:
# """ train with baum-welch """
# for i, file_obj in enumerate(file_data): 
#     sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
#     train_read(file_obj)

In [27]:
p = Pool(args["ncores"])

In [28]:
print("Prediction: ")
results = []
try:
    for i, res in enumerate(p.imap_unordered(basecall_read, file_data), 1):
        results.append(res)
        sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
    p.close()
    p.join()
except KeyboardInterrupt:
    p.terminate()

Prediction: 
done 100.000000%

### Stats

In [29]:
types = ["metrichor", "called", "random"]
fasta_files = {t: "{0}.{1}.fa".format(args["out_basename"], t) for t in types}

In [30]:
## metrichor fasta
with open(fasta_files["metrichor"], 'w') as f: 
    for file_obj in file_data: 
        f.write(">ch{0}_file{1}_metrichor".format(file_obj["channel"], file_obj["file_id"])+ "\n")
        f.write(file_obj["fastq"].split("\n")[1] + "\n")

In [31]:
## called fasta/random fasta
with open(fasta_files["called"], 'w') as f: 
    with open(fasta_files["random"], 'w') as fr:
        for channel, file_id, seq in results: 
            f.write(">ch{0}_file{1}_called".format(file_obj["channel"], file_obj["file_id"])+ "\n")
            fr.write(">ch{0}_file{1}_random".format(file_obj["channel"], file_obj["file_id"])+ "\n")
            f.write(seq + "\n")
            fr.write("".join([random.choice("ACGT") for _ in range(len(seq))]))

In [32]:
for t in types: 
    sam_file = "{0}.{1}.sam".format(args["out_basename"], t)
    graphmap(ref_file, fasta_files[t], sam_file, args["ncores"])
    prepare_sam("{0}.{1}".format(args["out_basename"], t))

[Index 16:38:25] Running in fast and sensitive mode. Two indexes will be used (double memory consumption).
[Index 16:38:25] Index already exists. Loading from file.
[Index 16:38:26] Secondary index already exists. Loading from file.
[Index 16:38:27] Index loaded in 0.62 sec.
[Index 16:38:27] Memory consumption: [currentRSS = 516 MB, peakRSS = 4501 MB]

[Run 16:38:27] Automatically setting the maximum allowed number of regions: max. 500, attempt to reduce after 100
[Run 16:38:27] Reference genome is assumed to be linear.
[Run 16:38:27] Only one alignment will be reported per mapped read.
[ProcessReads 16:38:27] Reads will be loaded in batches of up to 200 MB in size.
[ProcessReads 16:38:27] Batch of 1802 reads (9 MiB) loaded in 0.08 sec. (34505912 bases)
[ProcessReads 16:38:27] Memory consumption: [currentRSS = 526 MB, peakRSS = 4501 MB]
[ProcessReads 16:38:27] Using 62 threads.
[ProcessReads 16:38:43] [CPU time: 586.27 sec, RSS: 571 MB] Read: 1802/1802 (100.00%) [m: 1731, u: 71]       

In [33]:
def mk_stat(t):
    samfile = "{0}.{1}.sorted.bam".format(args["out_basename"], t)
    sst = samstats(samfile, ref, ncores=args["ncores"])
    return pandas.DataFrame(sst.print_summary())

In [34]:
stats = map(mk_stat, types)
print(types)
side_by_side(*stats)

['metrichor', 'called', 'random']


,0,1,2,3
0,mapped_reads/total_reads,1731,1802,96.059933%
1,significant_reads/total_reads,1531,1802,84.961154%
2,mapped_nts/total_nts,9181212,10421081,88.102300%
3,editdistance/alignment_length,4188258,10465915,40.018078%
4,alignment_score/alignment_length,11856757,10465915,113.289254%
5,SNPs/mapped_nts,1731521,9181212,18.859395%
6,ins/mapped_nts,1169947,9181212,12.742838%
7,del/mapped_nts,1284703,9181212,13.992739%
,0,1,2,3
0,mapped_reads/total_reads,1718,1802,95.338513%


In [35]:
for t, df in zip(types, stats):
    with open("{0}.stats.{1}.html".format(args["out_basename"], t), 'w') as f:
        f.write(df.to_html())

In [36]:
# def score_consensus(t):
#     consensus = mk_consensus("{0}.{1}.sorted.bam".format(args["out_basename"], t), ref_file)
#     return(consensus)
#     consensus = consensus.split("\n")[1].to_upper()
#     score = needle(ref, consensus)
#     return (consensus, score)

In [37]:
# p = Pool(args["ncores"])
# try:
#     consensus = p.map(score_consensus, types)
#     p.close()
# except KeyboardInterrupt:
#     p.terminate()

In [38]:
# consensus

In [39]:
# mk_consensus("{0}.{1}.sorted.bam".format(args["out_basename"], "metrichor"), ref_file)